# Install

In [1]:
import os
os.chdir('./GymGo-master')

In [2]:
! pip install -e .

Obtaining file:///Users/yuetongliu/Desktop/Reinforcement_Learning_on_Go/Code/14_02_24_Go/GymGo-master
  Attempting uninstall: gym-go
    Found existing installation: gym-go 0.0.1
    Uninstalling gym-go-0.0.1:
      Successfully uninstalled gym-go-0.0.1
  Running setup.py develop for gym-go


In [3]:
import gym
import random
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [216]:
# Program
class Learn(object):

    # board size, value function, action function
    def __init__(self, size):
        self.size = size
        # define state value function as a dict
        # key is position of b and w
        # value is state value (a number)
        self.value_function = dict()
        
        # add new state if not exist
        #if tuple(go_env.state()[0:2].reshape(size**2*2)) not in value_function: 
        #    value_function[tuple(go_env.state()[0:2].reshape(size**2*2))] = 0
        
        # define action value function as a dict
        # key is position of b and w
        # value is action value (an array)
        self.action_function = dict()
        
        #if tuple(go_env.state()[0:2].reshape(size**2*2)) not in action_function:
        #    action_function[tuple(go_env.state()[0:2].reshape(size**2*2))] = np.zeros(size**2+1)
        
        self.env = gym.make('gym_go:go-v0', size=self.size, reward_method = "heuristic")
        
        #self.state = self.env.state()[0:2].reshape(size**2*2)
    
    # helper
    def intersection(self,lst1, lst2):
        lst3 = [value for value in lst1 if value in lst2]
        return lst3

    ## greedy policy
    def apply_policy(self, state, epsilon):
        """
        Apply the policy of the agent
        Args:
            state: tuple of length 2
            epsilon: exploration probability, 0 for greedy behavior, 1 for pure exploration
        Returns:
            the selected action for the state under the current policy
        """
        ## random
        if np.random.uniform(0, 1) < epsilon:
            action_index = self.env.uniform_random_action()
        else: 
            # if not in action function, build a new one
            if tuple(state) not in self.action_function:
                self.action_function[tuple(state)] = np.zeros(size**2+1)
            
        
            # actions with max value
            greedy_action_value = np.max(self.action_function[tuple(state)])
            greedy_indices = [i for i, a in enumerate(self.action_function[tuple(state)]) if
                          a == greedy_action_value]
            
            valid = np.nonzero(self.env.valid_moves())[0]
            
            # overlap between max and valid action 
            idx = self.intersection(greedy_indices, valid)
            if idx == []:
                action_index = size**2
            #print(greedy_indices)
            else:
                action_index = np.random.choice(self.intersection(greedy_indices, valid))
        
        
        return action_index
    
    def sarsa_td(self, n_episodes=1000, alpha=0.01, gamma=0.9):
        """
        Run the sarsa control algorithm (TD0), finding the optimal policy and action function
        :param n_episodes: int, amount of episodes to train
        :param alpha: learning rate
        :param gamma: discount factor of future rewards
        :return: finds the optimal policy for move chess
        """
        
        for k in range(n_episodes):
            self.env = gym.make('gym_go:go-v0', size=self.size, reward_method = "heuristic")
            
            done = False
            epsilon = max(1 / (1 + k), 0.05)
            
            while not done:
                # current state - make action
                state = self.env.state()[0:2].reshape(size**2*2)
                action_index = self.apply_policy(state,epsilon)
                
                successor_state, reward, done, info = self.env.step(action_index)
                successor_state = successor_state[0:2].reshape(size**2*2)
                
                # successor state - make action
                if tuple(successor_state) not in self.action_function:
                    self.action_function[tuple(successor_state)] = np.zeros(size**2+1)
                
                # current state= get action value
                if tuple(state) not in self.action_function:
                    self.action_function[tuple(state)] = np.zeros(size**2+1)
                    action_value = 0
                else:
                    action_value = self.action_function[tuple(state)][action_index]
                    
                #successor state - get action value
                successor_action_index = self.apply_policy(successor_state, epsilon)

                successor_action_value = self.action_function[tuple(successor_state)][successor_action_index]
                    
                

                # update action value function
                q_update = alpha * (reward + gamma * successor_action_value - action_value)
                self.action_function[tuple(state)][action_index] += q_update
                
                if done:
                    break
                
                # white
                action = self.env.uniform_random_action()
                state, reward, done, info = self.env.step(action)
        

In [222]:
# train
a = Learn(size = 3)
a.sarsa_td(n_episodes=1000)

In [264]:
# after traning 
rewards = []

for i in range(1000): 
    
        a.env = gym.make('gym_go:go-v0', size=size, reward_method = 'real')
    
        
            
        # number of stone
        # number of black and white
        # difference
        
        
        
        done = False
          
        while not done:
            state = a.env.state()[0:2].reshape(size**2*2)
            action_index = a.apply_policy(state, 0.1)
            #action = go_env.uniform_random_action()
            state, reward, done, info = a.env.step(action_index)
            
            if done:
                break
            

        # white    
            action = a.env.uniform_random_action()
            state, reward, done, info = a.env.step(action)
            
        
        rewards.append(reward)

In [266]:
rewards.count(1.0),rewards.count(0),rewards.count(-1)

(609, 49, 342)

In [271]:
# before training 
rewards1 = []
for i in range(1000): 
    
        a.env = gym.make('gym_go:go-v0', size=size, reward_method = 'real')
            
        
        done = False
        while not done:
             
            action = a.env.uniform_random_action()
            state, reward, done, info = a.env.step(action)
            
        
        rewards1.append(reward)

In [272]:
rewards1.count(1.0),rewards1.count(0),rewards1.count(-1)

(520, 109, 371)

In [ ]:
TODO: reduce state by mirroring, hyperparameter tuning, draw value function

In [274]:
# 8 same symmetries
def all_symmetries(image):
    """
    :param image: A (C, BOARD_SIZE, BOARD_SIZE) numpy array, where C is any number
    :return: All 8 orientations that are symmetrical in a Go game over the 2nd and 3rd axes
    (i.e. rotations, flipping and combos of them)
    """
    symmetries = []

    for i in range(8):
        x = image
        if (i >> 0) % 2:
            # Horizontal flip
            x = np.flip(x, 2)
        if (i >> 1) % 2:
            # Vertical flip
            x = np.flip(x, 1)
        if (i >> 2) % 2:
            # Rotation 90 degrees
            x = np.rot90(x, axes=(1, 2))
            
        symmetries.append(x)

    return symmetries

In [318]:
a.env = gym.make('gym_go:go-v0', size=size, reward_method = 'real')

state, reward, done, info = a.env.step(3)
state, reward, done, info = a.env.step(4)
#state[0]
state[0]-state[1]
#all_symmetries(state)
all_symmetries((state[0]-state[1]).reshape(1,3,3))[1]



array([[[ 0.,  0.,  0.],
        [ 0., -1.,  1.],
        [ 0.,  0.,  0.]]])